In [ ]:
import pyoxigraph as ox
import pandas as pd

In [ ]:
store = ox.Store("data/wety.db")

In [ ]:
def get_children(root):
    return [(w['child'].value, w['lang'].value, w['term'].value) for w in store.query(
        f"""SELECT ?child ?lang ?term WHERE {{ 
            ?sourceNode <p:item> <{root}> .
            ?child <p:source> ?sourceNode .
            ?child <p:lang> ?lang .
            ?child <p:term> ?term .
        }}"""
    )]

def get_descendants(r):
    descendants = {}
    children = get_children(r)
    for child in children:
        descendants["{}, {}".format(child[1], child[2])] = get_descendants(child[0])
    return descendants

In [ ]:
root = "sekʷ-"
r = [s['w'].value for s in store.query(f'SELECT ?w WHERE {{ ?w <p:term> "{root}" . ?w <p:lang> "ine-pro" }}')][0]
d = get_descendants(r)

In [ ]:
d

In [ ]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <p:glossNum> ?glossNum .
            ?item <p:glossNum> ?glossNum .
            ?item <p:lang> ?lang .
            ?item <p:pos> ?pos .
            ?item <p:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum.csv", index=False)
print(len(data))

In [ ]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <p:glossNum> ?glossNum .
            ?item <p:etyNum> ?etyNum .
            ?item <p:lang> ?lang .
            ?item <p:pos> ?pos .
            ?item <p:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum_and_etyNum.csv", index=False)
print(len(data))